# Home assignment #2

**Author: Márton Nagy**

**Course: Web Scraping with Python**

In my second home assignment, I have decided to collect data on Interpol red and yellow notices from 2024. Red notices are basically international arrest warrants, while yellow notices are issued for missing people. After getting the data, I organize it into two tables: a table for notices, and a table for warrants (as one red notice can have multiple warrants tied to it (but there are no warrants for yellow notices)).

Detailed (though sometimes not quite correct in practice) description on the Interpol API can be found here: https://interpol.api.bund.dev/

**This relates to the following task:**

**API Selection Find an API that provides JSON responses. Choose an API that you find interesting and suitable for this assignment.**

First, we have to import the necessary packages.

Note that for tqdm to work correctly in my local environment, I had to:
- pip install ipywidgets
- jupyter labextension install @jupyter-widgets/jupyterlab-manager
- pip install --upgrade ipywidgets

In [3]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from scrapethat import *
from plotnine import *
import warnings
%matplotlib inline
warnings.filterwarnings("ignore")
import json
from tqdm.notebook import tqdm
import itertools

Then, we define a few functions. Note, that I had to add some additional information to the headers of the call, as the API blocked me if the code did not simulate that the call is from a web browser.

A very weird problem I've run into with this API that there is a hard cap of 160 results returned at maximum (not by page, but by total entries returned). These either fit to one page, or multiple pages if requested so - but pagination only works within the 160 returned results. As I wanted to get practically all notices, I had to find a workaround for this. My idea was to make very specific API calls (by country, sex and age range) so that the result fits into this 160 result cap. This feature/bug is not mentioned anywhere in the linked documentation, so I only figured this out by trial and error.

The next function makes an API call to a specific segment by country, sex, age and notice type. I will later use these IDs to get more detailed data from another API endpoint. If getting the data fails for some reason, the function returns with None. **This relates to the task: Create a list of input parameters (e.g., Pokémon names, city names).**

Note that docstrings for all of the functions were generated through an LLM service.

In [6]:
def get_notice_id_segment(type, parameter_list):
    """
    Fetches and returns a list of notice IDs from Interpol's public notices API based on specified criteria.

    Parameters:
    type (str): The type of notice to retrieve ('red' or 'yellow').
    parameter_list (list): A list of parameters containing:
        - parameter_list[0] (str): Nationality (e.g., 'US', 'FR').
        - parameter_list[1] (int): Minimum age for the notice search.
        - parameter_list[2] (int): Age increment to determine the maximum age.
        - parameter_list[3] (str): Sex identifier ('M' for male, 'F' for female, 'U' for unidentified).

    Returns:
    list: A list of entity IDs (str) if the API call is successful.
    None: Returns None if an exception occurs during the API request or data parsing.

    Notes:
    - A warning is printed if the total number of results exceeds 160, suggesting narrower search criteria.
    - Ensure `requests` and `json` libraries are imported for this function to work.
    - The function uses a generic `try-except` block, so any exception (e.g., network issues, invalid response) will return `None`.
    """

    headers = {
        'accept': 'application/json',
        'accept-language': 'en-US,en;q=0.9',
        'cache-control': 'max-age=0',
        'dnt': '1',
        'priority': 'u=0, i',
        'sec-ch-ua': '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'none',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
    }
    params = { #the parameters are taken as a list, to make the map later more simple
        'nationality': parameter_list[0],
        'ageMax': str(parameter_list[1] + parameter_list[2]),
        'ageMin': str(parameter_list[1]),
        'sexId': parameter_list[3],
        'page': '1',
        'resultPerPage': '200'
    }
    try:
        response = requests.get(f'https://ws-public.interpol.int/notices/v1/{type}', params=params, headers=headers)
        result = json.loads(response.text)
        t = result['_embedded']['notices']
        if int(result['total']) > 160:
            print(f'Warning: Result number is larger than 160 for country: {nationality}, sex: {sex}, age: {age} - {age + age_increment}, type: {type}.')
            print('Please consider using narrower categories!')
        ids = [x['entity_id'] for x in t]
        return ids
    except:
        return None

I've decided to get data on  G7 countries, all sexes and ages from 0 to 120, in 5-year age groups.

In [8]:
country_codes = ['CA', 'FR', 'DE', 'IT', 'JP', 'GB', 'US']
sexids = ['M', 'F', 'U']
from_age = 0
to_age = 120
age_increment = 5

Then, I create a list of lists for all the possible combinations of the above paramters. This is needed so that we can use a map to get all the needed IDs.

In [10]:
params = [[country, age, age_increment-1, sex] for country in country_codes for sex in sexids for age in range(from_age, to_age, age_increment)]
len(params)

504

Now we can map our function to the parameter list of lists. We will get back a list of entity IDs for all the possible parameter combinations, so the notice_ids dictionary will contain two list of lists. I've also included a tqdm progressbar to show the process. (This might take a few minutes to run. Note that I've experimented with concurrent requests to speed up but I got rate limited very quickly so I stuck with this slower solution.)

As we get no warnings printed out, we can be sure that we got all the notices in the given categories. If there would be warnings, setting the age increment to a smaller number may help.

In [12]:
notice_ids = {
    'yellow' : list(map(lambda p: get_notice_id_segment('yellow', p), 
                        tqdm(params, desc="Fetching yellow notice IDs"))),
    'red' : list(map(lambda p: get_notice_id_segment('red', p), 
                     tqdm(params, desc="Fetching red notice IDs")))
}

Fetching yellow notice IDs:   0%|          | 0/504 [00:00<?, ?it/s]

Fetching red notice IDs:   0%|          | 0/504 [00:00<?, ?it/s]

Now we should flatten out the lists, filter duplicates (there might be some, as a person may have multiple nationalities), and also filter out None values (which can be a result of a failed API call). **This relates to the task: Create a list of input parameters (e.g., Pokémon names, city names).**

In [14]:
notice_ids = {
    'yellow' : [item for item in list(set(itertools.chain.from_iterable(notice_ids['yellow']))) if item is not None],
    'red' : [item for item in list(set(itertools.chain.from_iterable(notice_ids['red']))) if item is not None]
}

In [15]:
print(len(notice_ids['yellow']))
print(len(notice_ids['red']))

576
233


The next function takes a notice type and an ID and returns the detailed data in a dictionary by making an API call to a different end point. There is also some basic error handling built-in so that timeout errors do not break the code. These kind of errors only happened to me if I was running the code on an unstable internet connection.

Note that the forward slash (/) sign in the notice ID has to be replaced %2F, the standard HTML URL encoding of the forward slash sign for the API call to work.

**This relates to the following task:**

**Write a function that:**
- **Takes one input parameter (e.g., Pokémon name, city name, or product ID).**
- **Calls the API with the input parameter.**
- **Processes the JSON response to extract meaningful data.**
- **Returns a simple dictionary with the extracted data.**

In [17]:
def get_one_notice(type, notice_id):
    """
    Retrieves detailed information about a specific Interpol notice by its ID.

    Parameters:
    type (str): The type of notice to retrieve ('red' or 'yellow').
    notice_id (str): The unique identifier of the notice. Slashes ('/') in the ID are URL-encoded.

    Returns:
    dict: A dictionary containing detailed information about the notice, with an added 'type' key to indicate the notice type.
    None: Returns None if an exception occurs during the API request or data parsing.

    Notes:
    - Adds a 'type' key to the resulting dictionary to include the notice type for easier classification.
    - The function handles potential exceptions with a generic `try-except` block, returning None on failure.
    - Ensure `requests` and `json` libraries are imported for this function to work.
    """

    headers = {
        'accept': 'application/json',
        'accept-language': 'en-US,en;q=0.9',
        'cache-control': 'max-age=0',
        'dnt': '1',
        'priority': 'u=0, i',
        'sec-ch-ua': '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'none',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
    }
    
    try:
        response = requests.get(f'https://ws-public.interpol.int/notices/v1/{type}/{notice_id.replace('/','%2F')}', headers=headers)
        result_dict = json.loads(response.text)
    except:
        return None

    result_dict['type'] = type #adding type (red or yellow) to the dictionary so that they fit into one list
    return result_dict

Now we can map our IDs to the above function by type. In the end, we will have a list of resulting dictionaries, one dictionary on each notice.

Note that this also takes a few minutes to run, but there is a progress bar to show how far the code has progressed. **This relates to the following task: Use the map() function to apply your function to this list of inputs. Avoid using explicit for loops where possible.**

In [20]:
notices_list = list(map(lambda notice_id: get_one_notice('yellow', notice_id), 
                        tqdm(notice_ids['yellow'], desc="Processing yellow notices")))

notices_list.extend(list(map(lambda notice_id: get_one_notice('red', notice_id), 
                             tqdm(notice_ids['red'], desc="Processing red notices"))))

Processing yellow notices:   0%|          | 0/576 [00:00<?, ?it/s]

Processing red notices:   0%|          | 0/233 [00:00<?, ?it/s]

Again, we have to filter out None values from the list that might have resulted if some of the API calls failed.

In [22]:
notices_list = [item for item in notices_list if item is not None]
len(notices_list)

809

Now let's write the results to the disk so that we do not have to bother with making the API calls all over again of we want to come back to this project.

In [24]:
with open('notices_list.json', 'w') as f:
    json.dump(notices_list, f)

Reading the file from the disk.

In [26]:
with open('notices_list.json', 'r') as f:
  data = json.load(f)

We can try to convert the results to a DataFrame - but as we can see, this does not look so good because of lists and nested dictionaries. So instead, we should parse the results manually. **This relates to task: Convert the list of dictionaries (output from map) into a Pandas DataFrame.**

In [28]:
pd.DataFrame(data).head().T

,0,1,2,3,4
country,FR,EC,GB,US,HK
date_of_birth,1966/10/04,2009/06/18,1980/11/29,1969/06/28,1978/01/28
mother_name,LARUE,OCHOCKY ROSSERO,Hopwood,None,None
countries_likely_to_be_visited,None,[US],None,"[US, CA, BS]","[IN, HK, CN]"
mother_forename,CLAIRE HENRIETTE,ILSE,Shirley,None,None
nationalities,[FR],[US],[GB],"[CA, US]",[CA]
eyes_colors_id,[BRO],[BLA],[BLU],[BLU],[BRO]
sex_id,F,F,F,F,F
forename,ISABELLE ALINE MICHELLE,MINETTE,DONNA MARIE,BRITT ASLO,ANI
country_of_birth_id,FR,US,GB,AU,CA


This function takes one notice and returns a nicely parsed dictionary with all the needed elements in it. **This (and also below cells) relate to the following task: Ensure the DataFrame is well-structured, with appropriate column names.**

In [30]:
def parse_one_notice(notice):
    """
    Parses a detailed Interpol notice and extracts key information into a structured dictionary.

    Parameters:
    notice (dict): A dictionary containing detailed information about an Interpol notice.

    Returns:
    dict: A dictionary with parsed notice information, including identity, birth details, appearance, 
          nationality, country info, event info, and notice type. Missing values are set to None.

    Parsed Fields:
    - Identity:
        'entity_id', 'name', 'birth_name', 'forename', 'mother_forename', 'mother_name', 
        'father_forename', 'father_name'
    - Birth:
        'date_of_birth', 'place_of_birth', 'country_of_birth_id'
    - Appearance:
        'sex_id', 'height', 'weight', 'eyes_colors_id' (joined by '#'), 'hairs_id' (joined by '#'),
        'distinguishing_marks'
    - Nationality and Languages:
        'nationalities' (joined by '#'), 'languages_spoken_ids' (joined by '#')
    - Country Info:
        'countries_likely_to_be_visited' (joined by '#'), 'issuing_country', 'country'
    - Event Info:
        'date_of_event', 'place'
    - Notice Type:
        'type'

    Helper Functions:
    - get_value(key): Returns the value of `key` from `notice` or None if missing or set to 0.
    - join_list(key): Joins list values for `key` using ' # ' as a delimiter or returns None if not a list.

    Notes:
    - Fields that are lists (e.g., 'eyes_colors_id', 'nationalities') are concatenated using ' # '.
    - If the field is missing or empty, it is set to None for consistency.
    """

    def get_value(key): #get the value from the notice if the key exists, else set it to None
        value = notice.get(key, 0)
        return None if value == 0 else value

    def join_list(key): #join a list by # characters if the value of a key is a list, else set it to None
        value = notice.get(key, 0)
        if isinstance(value, list):
            return ' # '.join(str(item) for item in value if item is not None)
        return None

    notice_dict = {
        #identity
        'entity_id': get_value('entity_id'),
        'name': get_value('name'),
        'birth_name': get_value('birth_name'),
        'forename': get_value('forename'),
        'mother_forename': get_value('mother_forename'),
        'mother_name': get_value('mother_name'),
        'father_forename': get_value('father_forename'),
        'father_name': get_value('father_name'),

        #birth
        'date_of_birth': get_value('date_of_birth'),
        'place_of_birth': get_value('place_of_birth'),
        'country_of_birth_id': get_value('country_of_birth_id'),
        
        #appearance
        'sex_id': get_value('sex_id'),
        'height': get_value('height'),
        'weight': get_value('weight'),
        'eyes_colors_id': join_list('eyes_colors_id'),
        'hairs_id': join_list('hairs_id'),
        'distinguishing_marks': get_value('distinguishing_marks'),

        #nationality
        'nationalities': join_list('nationalities'),
        'languages_spoken_ids': join_list('languages_spoken_ids'),

        #country info
        'countries_likely_to_be_visited': join_list('countries_likely_to_be_visited'),
        'issuing_country': get_value('issuing_country'),
        'country': get_value('country'),
        
        #event info
        'date_of_event': get_value('date_of_event'),
        'place': get_value('place'),

        #notice type
        'type': get_value('type')
    }

    return notice_dict


This function takes a notice and returns a list of the warrants associated with it, while it also adds to every warrant the entity ID associated with it. This is needed so that we can connect the warrants and the notices tables together.

In [32]:
def parse_warrants(notice):
    """
    Extracts and parses arrest warrants from an Interpol notice.

    Parameters:
    notice (dict): A dictionary containing detailed information about an Interpol notice, including arrest warrants.

    Returns:
    list: A list of dictionaries, each representing an arrest warrant. Each warrant dictionary is augmented 
          with the 'entity_id' from the notice. Returns an empty list if no warrants are present or if the data is not a list.

    Notes:
    - If 'arrest_warrants' is missing or not a list, an empty list is returned.
    - The 'entity_id' from the notice is added to each warrant for contextual linking.
    """

    warrants = notice.get('arrest_warrants', 0)
    return [{**warrant, 'entity_id': notice['entity_id']} for warrant in warrants] if isinstance(warrants, list) else []


Now we can map our parsing function to our JSON data.

In [34]:
notices_parsed = list(map(parse_one_notice, data))

And the resulting DataFrame looks more or less all right.

In [36]:
notices_df = pd.DataFrame(notices_parsed)

In [37]:
notices_df

,entity_id,name,birth_name,forename,mother_forename,mother_name,father_forename,father_name,date_of_birth,place_of_birth,...,hairs_id,distinguishing_marks,nationalities,languages_spoken_ids,countries_likely_to_be_visited,issuing_country,country,date_of_event,place,type
0,2023/34189,DUBOIS,DUBOIS,ISABELLE ALINE MICHELLE,CLAIRE HENRIETTE,LARUE,LUCIEN JOSEPH,DUBOIS,1966/10/04,LYON,...,GRYG,TATOUAGE SEIN GAUCHE (VOIR PHOTO).\r\nCHEVEUX ...,FR,FRE,None,FR,FR,2022/12/31,ROCHECHOUART,yellow
1,2012/320974,ROSSERO,ROSSERO,MINETTE,ILSE,OCHOCKY ROSSERO,DIEGO FERNANDO,ROSERO JHAYYA,2009/06/18,FLORIDA,...,BRO,None,US,ENG,US,EC,EC,2012/07/16,QUITO,yellow
2,2017/200566,KEOGH,None,DONNA MARIE,Shirley,Hopwood,Brian,Keogh,1980/11/29,Middlesborough,...,YELB,Scars - \r\nRight abdomen - Appendix operation...,GB,ENG,None,GB,GB,1998/05/01,Middlesborough,yellow
3,2021/50095,TAYLOR,ROLFE,BRITT ASLO,None,None,None,None,1969/06/28,None,...,RED,None,CA # US,ENG,US # CA # BS,US,US,2021/06/02,None,yellow
4,2017/134336,ASHEKIAN,None,ANI,None,None,None,None,1978/01/28,None,...,BRO,"'Ld' faded tattoo on her bikini line, on her l...",CA,ENG,IN # HK # CN,HK,HK,2008/11/10,None,yellow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,2009/16041,BADO,None,GRIGORY,None,None,None,None,1949/02/25,MOSCOW,...,None,None,US # RU,None,None,None,None,None,None,red
805,2015/56262,LYTUS,None,JOHN,None,None,None,None,1963/08/06,None,...,BLA,None,US,ENG # SPA,None,None,None,None,None,red
806,2019/41200,HAQ,None,MASOOD UL,None,None,None,None,1973/09/10,"MIR PUR, AZAD JAMMU & KASHMIR",...,BLA,(1) Black Mole on left Cheek\r\n(2)\tCut M...,GB # PK,ENG # URD,None,None,None,None,None,red
807,2024/19162,CHIBOUNI,None,WISSEM,None,None,None,None,2005/01/29,THONON-LES-BAINS,...,None,None,FR,FRE,None,None,None,None,None,red


The API call sometimes results in an error (though it gives back a JSON object), which is indicated by the name being Error and all other fields being None. We should filter these out. Again, these kind of errors only happened to me on an unstable internet connection.

In [39]:
notices_df[notices_df['entity_id'].isna()]

,entity_id,name,birth_name,forename,mother_forename,mother_name,father_forename,father_name,date_of_birth,place_of_birth,...,hairs_id,distinguishing_marks,nationalities,languages_spoken_ids,countries_likely_to_be_visited,issuing_country,country,date_of_event,place,type


In [40]:
notices_df = notices_df[notices_df['name'] != 'Error']

We can also parse the warrants and make a DataFrame out of it. Charge translation has been dropped as it was all null values.

In [42]:
warrants_parsed = [warrant for notice in data for warrant in parse_warrants(notice)]

In [43]:
warrants_df = pd.DataFrame(warrants_parsed)
warrants_df.charge_translation.isna().sum() == warrants_df.shape[0]

True

In [44]:
warrants_df = warrants_df.filter(['entity_id', 'issuing_country_id', 'charge'])
warrants_df

,entity_id,issuing_country_id,charge
0,2006/22280,US,(1) Forcible lewd act upon a child; \r\n(2) L...
1,2014/66413,US,Super Aggravated Sexual Assault Child of a Und...
2,2014/66413,US,Aggravated Sexual Assault of a Child Under 14
3,2004/39515,US,1) Conspiracy to commit murder \r\n2) Attempte...
4,2015/13691,IN,Dealing (arranging and sending abroad) in Narc...
...,...,...,...
277,2009/16041,RU,FRAUD PERFORMED IN LARGE AMOUNT
278,2015/56262,NI,VIOLACION AGRAVADA Y ROBO AGRAVADO
279,2019/41200,PK,"Planning, Terrorist Activities, Nefarious Desi..."
280,2024/19162,FR,*Tentative de meurtre


The only thing left to do data-cleaning wise is to convert the date columns into actual date object. First, I create a flag variable indicating whether date of birth is present only with a year, missing month and date. Note that in the case of G7 countries, there is actually no such birth date in the data (but when I experimented with other countries as well, this was an issue).

In [46]:
notices_df['date_of_birth_missing_md'] = notices_df['date_of_birth'].apply(lambda x: 1 if isinstance(x, str) and len(x) == 4 else 0)

Then we can convert the date of birht column, if only a year is given, then we set it to 1 January that year.

In [48]:
notices_df['date_of_birth'] = pd.to_datetime(
    notices_df['date_of_birth'].apply(lambda x: f"{x}/01/01" if isinstance(x, str) and len(x) == 4 else x), format='%Y/%m/%d',
    errors='coerce'
)

We can also convert the date of event field.

In [50]:
notices_df['date_of_event'] = pd.to_datetime(notices_df['date_of_event'], format='%Y/%m/%d', errors = 'coerce')

Now we have two nicely formatted DataFrames, ready for further analysis!

In [52]:
notices_df

,entity_id,name,birth_name,forename,mother_forename,mother_name,father_forename,father_name,date_of_birth,place_of_birth,...,distinguishing_marks,nationalities,languages_spoken_ids,countries_likely_to_be_visited,issuing_country,country,date_of_event,place,type,date_of_birth_missing_md
0,2023/34189,DUBOIS,DUBOIS,ISABELLE ALINE MICHELLE,CLAIRE HENRIETTE,LARUE,LUCIEN JOSEPH,DUBOIS,1966-10-04,LYON,...,TATOUAGE SEIN GAUCHE (VOIR PHOTO).\r\nCHEVEUX ...,FR,FRE,None,FR,FR,2022-12-31,ROCHECHOUART,yellow,0
1,2012/320974,ROSSERO,ROSSERO,MINETTE,ILSE,OCHOCKY ROSSERO,DIEGO FERNANDO,ROSERO JHAYYA,2009-06-18,FLORIDA,...,None,US,ENG,US,EC,EC,2012-07-16,QUITO,yellow,0
2,2017/200566,KEOGH,None,DONNA MARIE,Shirley,Hopwood,Brian,Keogh,1980-11-29,Middlesborough,...,Scars - \r\nRight abdomen - Appendix operation...,GB,ENG,None,GB,GB,1998-05-01,Middlesborough,yellow,0
3,2021/50095,TAYLOR,ROLFE,BRITT ASLO,None,None,None,None,1969-06-28,None,...,None,CA # US,ENG,US # CA # BS,US,US,2021-06-02,None,yellow,0
4,2017/134336,ASHEKIAN,None,ANI,None,None,None,None,1978-01-28,None,...,"'Ld' faded tattoo on her bikini line, on her l...",CA,ENG,IN # HK # CN,HK,HK,2008-11-10,None,yellow,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,2009/16041,BADO,None,GRIGORY,None,None,None,None,1949-02-25,MOSCOW,...,None,US # RU,None,None,None,None,NaT,None,red,0
805,2015/56262,LYTUS,None,JOHN,None,None,None,None,1963-08-06,None,...,None,US,ENG # SPA,None,None,None,NaT,None,red,0
806,2019/41200,HAQ,None,MASOOD UL,None,None,None,None,1973-09-10,"MIR PUR, AZAD JAMMU & KASHMIR",...,(1) Black Mole on left Cheek\r\n(2)\tCut M...,GB # PK,ENG # URD,None,None,None,NaT,None,red,0
807,2024/19162,CHIBOUNI,None,WISSEM,None,None,None,None,2005-01-29,THONON-LES-BAINS,...,None,FR,FRE,None,None,None,NaT,None,red,0


In [53]:
warrants_df

,entity_id,issuing_country_id,charge
0,2006/22280,US,(1) Forcible lewd act upon a child; \r\n(2) L...
1,2014/66413,US,Super Aggravated Sexual Assault Child of a Und...
2,2014/66413,US,Aggravated Sexual Assault of a Child Under 14
3,2004/39515,US,1) Conspiracy to commit murder \r\n2) Attempte...
4,2015/13691,IN,Dealing (arranging and sending abroad) in Narc...
...,...,...,...
277,2009/16041,RU,FRAUD PERFORMED IN LARGE AMOUNT
278,2015/56262,NI,VIOLACION AGRAVADA Y ROBO AGRAVADO
279,2019/41200,PK,"Planning, Terrorist Activities, Nefarious Desi..."
280,2024/19162,FR,*Tentative de meurtre


Note that we can easily merge the two tables. For this, we must be aware that entity IDs are unique only in a certain notice type (so red and yellow notices can have the same entity ID). Also, only red notices can have warrants attached to them. So, we can create unique IDs by appending 'r' or 'y' to the entity ID based on the notice type.

In [55]:
notices_df['entity_id'] = notices_df.apply(lambda x: (x['entity_id'] + 'r') if x['type'] == 'red' else (x['entity_id'] + 'y'), axis=1)

In [56]:
warrants_df['entity_id'] = warrants_df['entity_id'].apply(lambda x: f'{x}r')

In [57]:
interpol_merged = notices_df.merge(warrants_df, how = 'left', on = 'entity_id')
interpol_merged

,entity_id,name,birth_name,forename,mother_forename,mother_name,father_forename,father_name,date_of_birth,place_of_birth,...,languages_spoken_ids,countries_likely_to_be_visited,issuing_country,country,date_of_event,place,type,date_of_birth_missing_md,issuing_country_id,charge
0,2023/34189y,DUBOIS,DUBOIS,ISABELLE ALINE MICHELLE,CLAIRE HENRIETTE,LARUE,LUCIEN JOSEPH,DUBOIS,1966-10-04,LYON,...,FRE,None,FR,FR,2022-12-31,ROCHECHOUART,yellow,0,NaN,NaN
1,2012/320974y,ROSSERO,ROSSERO,MINETTE,ILSE,OCHOCKY ROSSERO,DIEGO FERNANDO,ROSERO JHAYYA,2009-06-18,FLORIDA,...,ENG,US,EC,EC,2012-07-16,QUITO,yellow,0,NaN,NaN
2,2017/200566y,KEOGH,None,DONNA MARIE,Shirley,Hopwood,Brian,Keogh,1980-11-29,Middlesborough,...,ENG,None,GB,GB,1998-05-01,Middlesborough,yellow,0,NaN,NaN
3,2021/50095y,TAYLOR,ROLFE,BRITT ASLO,None,None,None,None,1969-06-28,None,...,ENG,US # CA # BS,US,US,2021-06-02,None,yellow,0,NaN,NaN
4,2017/134336y,ASHEKIAN,None,ANI,None,None,None,None,1978-01-28,None,...,ENG,IN # HK # CN,HK,HK,2008-11-10,None,yellow,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853,2009/16041r,BADO,None,GRIGORY,None,None,None,None,1949-02-25,MOSCOW,...,None,None,None,None,NaT,None,red,0,RU,FRAUD PERFORMED IN LARGE AMOUNT
854,2015/56262r,LYTUS,None,JOHN,None,None,None,None,1963-08-06,None,...,ENG # SPA,None,None,None,NaT,None,red,0,NI,VIOLACION AGRAVADA Y ROBO AGRAVADO
855,2019/41200r,HAQ,None,MASOOD UL,None,None,None,None,1973-09-10,"MIR PUR, AZAD JAMMU & KASHMIR",...,ENG # URD,None,None,None,NaT,None,red,0,PK,"Planning, Terrorist Activities, Nefarious Desi..."
856,2024/19162r,CHIBOUNI,None,WISSEM,None,None,None,None,2005-01-29,THONON-LES-BAINS,...,FRE,None,None,None,NaT,None,red,0,FR,*Tentative de meurtre


It turns out, that for the downloaded notices, there are indeed some red notices with multiple warrants, e.g.:

In [59]:
interpol_merged[interpol_merged['entity_id'] == '2011/59946r'].T

,817,818,819,820
entity_id,2011/59946r,2011/59946r,2011/59946r,2011/59946r
name,BAUMAN,BAUMAN,BAUMAN,BAUMAN
birth_name,None,None,None,None
forename,MARK STEPHEN,MARK STEPHEN,MARK STEPHEN,MARK STEPHEN
mother_forename,None,None,None,None
mother_name,None,None,None,None
father_forename,None,None,None,None
father_name,None,None,None,None
date_of_birth,1961-02-15 00:00:00,1961-02-15 00:00:00,1961-02-15 00:00:00,1961-02-15 00:00:00
place_of_birth,"RED WING, MINNESOTA","RED WING, MINNESOTA","RED WING, MINNESOTA","RED WING, MINNESOTA"


Lastly, we can save the resulting merged DataFrame into a CSV file for further usage.

In [61]:
interpol_merged.to_csv('interpol_notices_G7_data.csv', index = False)